In [1]:
import numpy as np
import pandas as pd
import json
from jordan.model.Sample_MIL import InstanceModels, RaggedModels
from jordan.model.KerasLayers import Losses, Metrics
import tensorflow as tf
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from jordan.model import DatasetsUtils
import pickle
import logomaker
import matplotlib.pyplot as plt

#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[-4], True)
#tf.config.experimental.set_visible_devices(physical_devices[-4], 'GPU')

# Load the raw data files
D, tcga_maf, samples = pickle.load(open('/home/janaya2/Desktop/ATGC_paper/figures/tumor_classification/data/data.pkl', 'rb'))

cancerhotspots_df = pd.read_csv("/home/sahn33/Documents/cancerhotspots.v2.maf",sep="\t", low_memory=True) #usecols=["Chromosome","Start_Position", "End_Position", "Reference_Allele","Tumor_Seq_Allele2"],

with open("publication_hotspots.vcf", "r") as f:
    lines = f.readlines()
    chrom_index = [i for i, line in enumerate(lines) if line.strip().startswith("#CHROM")]
    data = lines[chrom_index[0]:]
header = data[0].strip().split("\t")
informations = [d.strip().split("\t") for d in data[1:]]
publication_hotspots_df = pd.DataFrame(informations, columns=header)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (4,15,40,41,43,44,74,83,84,85,86,87,97,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
# Create a dataframe that identifies the instances that are hotspots.
cancerhotspots_df.drop_duplicates(subset=["Chromosome","Start_Position", "End_Position", "Reference_Allele","Tumor_Seq_Allele2"],keep=False,inplace=True)

cancerhotspots_df = cancerhotspots_df[["Chromosome","Start_Position", "End_Position", "Reference_Allele","Tumor_Seq_Allele2"]] #
cancerhotspots_df["id"] = cancerhotspots_df.index

hotspot_df_ = pd.merge(cancerhotspots_df, tcga_maf, how='right', on=["Chromosome","Start_Position", "End_Position", "Reference_Allele","Tumor_Seq_Allele2"], suffixes=('_duplicate',''))

publication_hotspots_df.drop_duplicates(subset=["#CHROM","POS","REF","ALT"],keep=False,inplace=True)

hotspot_df_copy = hotspot_df_
hotspot_df_copy = hotspot_df_copy[["Chromosome","Start_Position","Ref","Alt","id"]] #
hotspot_df_copy['Alt'] = hotspot_df_copy['Alt'].apply(lambda x: x[:1])
hotspot_df_copy['Ref'] = hotspot_df_copy['Ref'].apply(lambda x: x[:1])
hotspot_df_copy['Start_Position'] = hotspot_df_copy['Start_Position'].apply(lambda x: str(x))
publication_hotspots_df["my_index"] = publication_hotspots_df.index
hotspot_df_merge_cols = ["Chromosome","Start_Position","Ref","Alt"]
publication_hotspots_df_merge_cols = ["#CHROM","POS","REF","ALT"]

hotspot_df = pd.merge(publication_hotspots_df, hotspot_df_copy, how='right', right_on = hotspot_df_merge_cols, left_on = publication_hotspots_df_merge_cols, suffixes=('_duplicate',''))


In [4]:
# Create a column in the dataframe that flags for InDel instances
tcga_maf['indel'] = np.where((tcga_maf['Variant_Classification'] == 'Frame_Shift_Del') |
                             (tcga_maf['Variant_Classification'] == 'Frame_Shift_Ins') |
                             (tcga_maf['Variant_Classification'] == 'In_Frame_Del') |
                             (tcga_maf['Variant_Classification'] == 'In_Frame_Ins'),
                             True, False)

In [5]:
# Re-assign labels from TCGA to custom NCIT labels and encode the data

strand_emb_mat = np.concatenate([np.zeros(2)[np.newaxis, :], np.diag(np.ones(2))], axis=0)
D['strand_emb'] = strand_emb_mat[D['strand']]

###
# filtering the NCI-T labels (https://livejohnshopkins-my.sharepoint.com/:x:/r/personal/abaras1_jh_edu/_layouts/15/doc2.aspx?sourcedoc=%7B5f92f0fc-ec6c-40d5-ab17-0d3345f9f2c2%7D&action=edit&activeCell=%27Sheet1%27!B21&wdinitialsession=e072a38f-57c8-4c1f-885b-efaefcc81d35&wdrldsc=2&wdrldc=1&wdrldr=AccessTokenExpiredWarning%2CRefreshingExpiredAccessT)
ncit_labels_kept = ['Muscle-Invasive Bladder Carcinoma','Infiltrating Ductal Breast Carcinoma',
                    'Invasive Lobular Breast Carcinoma','Cervical Squamous Cell Carcinoma',
                    'Colorectal Adenocarcinoma','Glioblastoma','Head and Neck Squamous Cell Carcinoma',
                    'Clear Cell Renal Cell Carcinoma','Papillary Renal Cell Carcinoma','Astrocytoma',
                    'Oligoastrocytoma','Oligodendroglioma','Hepatocellular Carcinoma','Lung Adenocarcinoma',
                    'Lung Squamous Cell Carcinoma','Ovarian Serous Adenocarcinoma','Adenocarcinoma, Pancreas',
                    'Paraganglioma','Pheochromocytoma','Prostate Acinar Adenocarcinoma','Colorectal Adenocarcinoma',
                    'Desmoid-Type Fibromatosis','Leiomyosarcoma','Liposarcoma','Malignant Peripheral Nerve Sheath Tumor',
                    'Myxofibrosarcoma','Synovial Sarcoma','Undifferentiated Pleomorphic Sarcoma',
                    'Cutaneous Melanoma','Gastric Adenocarcinoma','Testicular Non-Seminomatous Germ Cell Tumor',
                    'Testicular Seminoma','Thyroid Gland Follicular Carcinoma','Thyroid Gland Papillary Carcinoma',
                    'Endometrial Endometrioid Adenocarcinoma','Endometrial Serous Adenocarcinoma']
idx_filter = samples['NCI-T Label'].isin(ncit_labels_kept)
ncit_samples = samples.loc[idx_filter]
PCPG_ncit = ['Paraganglioma','Pheochromocytoma']
SARC_ncit = ['Desmoid-Type Fibromatosis','Leiomyosarcoma','Liposarcoma','Malignant Peripheral Nerve Sheath Tumor',
             'Myxofibrosarcoma','Synovial Sarcoma','Undifferentiated Pleomorphic Sarcoma']
TGCT_ncit = ['Testicular Non-Seminomatous Germ Cell Tumor','Testicular Seminoma']
ncit_samples.loc[ncit_samples['NCI-T Label'].isin(PCPG_ncit), 'NCI-T Label'] = 'PCPG'
ncit_samples.loc[ncit_samples['NCI-T Label'].isin(SARC_ncit), 'NCI-T Label'] = 'SARC'
ncit_samples.loc[ncit_samples['NCI-T Label'].isin(TGCT_ncit), 'NCI-T Label'] = 'TGCT'

#samples = ncit_samples
A = ncit_samples['NCI-T Label'].astype('category')
###

##each instance has an associated sample index so we need to group the instances for each sample together
##the sample dataframe had its index reset so the indexes start at 0 and are concurrent, if you subset the sample dataframe then you'll have to only generate indexes for those samples
indexes = [np.where(D['sample_idx'] == idx) for idx in np.arange(samples.shape[0])[idx_filter]]

##the sequence encoder has five inputs
five_p = np.array([D['seq_5p'][i] for i in indexes], dtype='object')
three_p = np.array([D['seq_3p'][i] for i in indexes], dtype='object')
ref = np.array([D['seq_ref'][i] for i in indexes], dtype='object')
alt = np.array([D['seq_alt'][i] for i in indexes], dtype='object')
strand = np.array([D['strand_emb'][i] for i in indexes], dtype='object')
hotspots = np.array([~pd.isna(hotspot_df["my_index"]).values[i] for i in indexes], dtype='object')
indels = np.array([tcga_maf['indel'].values[i] for i in indexes], dtype='object')

##when we evaluate the model we don't want to be dropping out any instances
five_p_loader_eval = DatasetsUtils.Map.FromNumpy(five_p, tf.int16)
three_p_loader_eval = DatasetsUtils.Map.FromNumpy(three_p, tf.int16)
ref_loader_eval = DatasetsUtils.Map.FromNumpy(ref, tf.int16)
alt_loader_eval = DatasetsUtils.Map.FromNumpy(alt, tf.int16)
strand_loader_eval = DatasetsUtils.Map.FromNumpy(strand, tf.float32)

#A = samples['type'].astype('category')
classes = A.cat.categories.values
##integer values for random forest
classes_onehot = np.eye(len(classes))[A.cat.codes]
y_label = classes_onehot

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [6]:
# Load the models' results.

test_idx, weights = pickle.load(open('/home/mlee276/Desktop/TCGA-ML-main/results/mil_contexts_weights.pkl', 'rb'))

sequence_encoder = InstanceModels.VariantSequence(6, 4, 2, [16, 16, 16, 16], fusion_dimension=256)
mil = RaggedModels.MIL(instance_encoders=[sequence_encoder.model], sample_encoders=[], output_dims=[y_label.shape[-1]], output_types=['other'], instance_layers=[512], mil_hidden=[256, 128, 64], attention_layers=[64, 16], dropout=.2, instance_dropout=.3, regularization=0, input_dropout=.4)

attention_weights = []
embedding_weights = []
idx_test_all = []
for idx_test, fold_weights in zip(test_idx, weights):
    mil.model.set_weights(fold_weights)
    ds_test = tf.data.Dataset.from_tensor_slices(((
                                                       five_p_loader_eval(idx_test),
                                                       three_p_loader_eval(idx_test),
                                                       ref_loader_eval(idx_test),
                                                       alt_loader_eval(idx_test),
                                                       strand_loader_eval(idx_test),
                                                   ),
    ))
    ds_test = ds_test.batch(len(idx_test), drop_remainder=False)
    attention_weights.extend(mil.attention_model.predict(ds_test).to_list())
    idx_test_all.extend(idx_test)
    embedding_weights.append(mil.model.get_weights())


In [7]:
# Get MIL predictions
test_idx, mil_predictions = pickle.load(open('/home/mlee276/Desktop/TCGA-ML-main/results/mil_contexts_predictions.pkl', 'rb'))
mil_predictions_labels = np.argmax(mil_predictions, axis=-1)
# Get true labels
y_strat = np.argmax(y_label, axis=-1)
correct = (y_strat[np.concatenate(test_idx)])

hotspots_reordered = hotspots[idx_test_all] ### can only call this once or else the order gets mixed. 
indels_reordered = indels[idx_test_all]

#idx = 0
#attention_weights_ = np.array(attention_weights[idx]).flatten()
#hotspot_attention_weights = np.expand_dims(attention_weights_, axis=0)[:,hotspots[idx]][0]
#non_hotspot_attention_weights = np.expand_dims(attention_weights_, axis=0)[:,~hotspots[idx]][0]

# Returns the position weight matrix of a specific cancer type and context sequence.
def get_position_weight_matrix(class_num, sequence):
    # Get indexes of correct predictions
    correct_predictions_idx = []
    for i in range(len(correct)):
        if correct[i] == mil_predictions_labels[i] and correct[i] == class_num:
            correct_predictions_idx.append(i)
            
    matrix = []
    for idx in correct_predictions_idx:
        sample_attention_weights = np.concatenate(attention_weights[idx])
        matrix.append(sequence[idx_test_all][idx][sample_attention_weights > .9][:, :, 0]) 
    
    return matrix

# Returns the attention weights, hotspot attention weights, and non-hotspot attention weights associated with a specifc cancer type and context sequence. 
def get_position_weight_hotspots(class_num, sequence):
    # Get indexes of correct predictions
    correct_predictions_idx = []
    for i in range(len(correct)):
        if correct[i] == mil_predictions_labels[i] and correct[i] == class_num:
            correct_predictions_idx.append(i)
            
    all_sample_attention_weights = []
    hotspot_attention_weights = []
    non_hotspot_attention_weights = []
    for idx in correct_predictions_idx:
        sample_attention_weights = np.concatenate(attention_weights[idx])
        
        hotspot_attention_weights.append(np.expand_dims(sample_attention_weights, axis=0)[:,hotspots_reordered[idx]][0])
        non_hotspot_attention_weights.append(np.expand_dims(sample_attention_weights, axis=0)[:,~hotspots_reordered[idx]][0])
        
        all_sample_attention_weights.append(sample_attention_weights)
    
    return all_sample_attention_weights, hotspot_attention_weights, non_hotspot_attention_weights

# Returns the attention weights, indel attention weights, and non-indel attention weights associated with a specifc cancer type and context sequence.
def get_position_weight_indels(class_num, sequence):
    # Get indexes of correct predictions
    correct_predictions_idx = []
    for i in range(len(correct)):
        if correct[i] == mil_predictions_labels[i] and correct[i] == class_num:
            correct_predictions_idx.append(i)
            
    all_sample_attention_weights = []
    indel_attention_weights = []
    non_indel_attention_weights = []
    for idx in correct_predictions_idx:
        sample_attention_weights = np.concatenate(attention_weights[idx])
        
        indel_attention_weights.append(np.expand_dims(sample_attention_weights, axis=0)[:,indels_reordered[idx]][0])
        non_indel_attention_weights.append(np.expand_dims(sample_attention_weights, axis=0)[:,~indels_reordered[idx]][0])
        
        all_sample_attention_weights.append(sample_attention_weights)
    
    return all_sample_attention_weights, indel_attention_weights, non_indel_attention_weights

In [17]:
# Produces a matrix of histograms representing the frequency of attention weights for hotspot instances and non-hotspot instances for all cancer types.
%matplotlib
fig,ax = plt.subplots(5,6)
for i in range(27):
    t_, h_, nh_ = get_position_weight_hotspots(i, five_p) 
    c1 = 0;c2 = 0;c3 = 0
    for n in t_:
        c1 += len(n)
    for n in h_:
        c2 += len(n)
    for n in nh_:
        c3 += len(n)

    perc_hotspot = str(np.round_(100*c2/c1, 3))

    bin_nums = 200
    hotspot_att_weights = []
    for weights in h_:
        hotspot_att_weights.extend(weights)
    non_hotspot_att_weights = []
    for weights in nh_:
        non_hotspot_att_weights.extend(weights)

    #fig, ax = plt.subplots(figsize=(12, 6))
    ax[i//6,i%6].hist(non_hotspot_att_weights, bins=bin_nums, label="non hotspots attention weights")
    ax[i//6,i%6].hist(hotspot_att_weights, bins=bin_nums, label="hotspots attention weights")
    ax[i//6,i%6].set_xlabel('attention weights')
    ax[i//6,i%6].set_ylabel('Count')
    ax[i//6,i%6].set_title(classes[i] + "; " + perc_hotspot + "%", fontsize=8)
#fig.legend()
fig.suptitle("Gene HotSpots Attention Weight Distribution (orange is hotspots)")
    #plt.show()

Using matplotlib backend: Qt5Agg


Text(0.5, 0.98, 'Gene HotSpots Attention Weight Distribution (orange is hotspots)')

In [ ]:
# Produces a matrix of histograms representing the frequency of attention weights for indel instances and non-indel instances for all cancer types.
%matplotlib
fig,ax = plt.subplots(5,6)
for i in range(27):
    t_, h_, nh_ = get_position_weight_hotspots(i,genes)
    c1 = 0;c2 = 0;c3 = 0
    for n in t_:
        c1 += len(n)
    for n in h_:
        c2 += len(n)
    for n in nh_:
        c3 += len(n)

    perc_hotspot = str(np.round_(100*c2/c1, 3))

    bin_nums = 200
    hotspot_att_weights = []
    for weights in h_:
        hotspot_att_weights.extend(weights)
    non_hotspot_att_weights = []
    for weights in nh_:
        non_hotspot_att_weights.extend(weights)

    #fig, ax = plt.subplots(figsize=(12, 6))
    ax[i//6,i%6].hist(non_hotspot_att_weights, bins=bin_nums, label="non indels attention weights")
    ax[i//6,i%6].hist(hotspot_att_weights, bins=bin_nums, label="indels attention weights")
    ax[i//6,i%6].set_xlabel('attention weights')
    ax[i//6,i%6].set_ylabel('Count')
    ax[i//6,i%6].set_title(classes[i] + "; " + perc_hotspot + "%", fontsize=8)
#fig.legend()
fig.suptitle("Gene InDel Attention Weight Distribution (orange is hotspots)")
    #plt.show()